# Intro Agents from SCratch

## 3 Levels

1. LLMs + functions in prompt
2. LLMs + structured outputs/function calling
3. Agent loop

### 1. LLMs + functions in prompt

In [1]:
from openai import OpenAI

client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "system", "content": "You are a helpful assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

get_response("Hi! What are the levels of agents?")

'Do you mean in AI? If so, a common progression of agent “levels” (by sophistication) is:\n\n- Simple reflex agents: Act on condition–action rules; no internal state.\n- Model-based reflex agents: Keep an internal state (a model) to handle partial observability.\n- Goal-based agents: Plan actions to achieve explicit goals.\n- Utility-based agents: Choose actions that maximize a utility function (trade-offs, uncertainty).\n- Learning agents: Improve their behavior over time by learning; can augment any of the above.\n\nIf you meant another domain (e.g., support tiers, real estate licensing, autonomy levels in robotics), tell me which and I’ll tailor the answer.'

In [2]:
def write_file(file_path, content):
    """Takes in a file path and content, and writes the content to the file"""
    with open(file_path, "w") as f:
        f.write(content)

def read_file(file_path):
    """Takes in a file path and returns the content of the file"""
    with open(file_path, "r") as f:
        return f.read()


def level1_agent_llm_and_functions_in_prompt(prompt):
    function_info = """
    def write_file(file_path, content):
        '''Takes in a file path and content, and writes the content to the file'''
        with open(file_path, "w") as f:
            f.write(content)

    def read_file(file_path):
        '''Takes in a file path and returns the content of the file'''
        with open(file_path, "r") as f:
            return f.read()

    """
    full_prompt_with_function_info = f"""
    Take this request from a user: {prompt}.
    If the request involves writing to a file or reading to a file,
    you can output a call to these functions which you have access to: 
    {function_info}.
    """
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
                  {"role": "user", "content": full_prompt_with_function_info}]
    )
    
    return response.choices[0].message.content
    
level1_agent_llm_and_functions_in_prompt("Write a file called 'test.txt' with the content 'Hello, world!'")    

'write_file("test.txt", "Hello, world!")'

In [3]:
exec('write_file("test.txt", "Hello, world!")')

## Level 2: LLMs + structured outputs/function calling

In [5]:
from pydantic import BaseModel, Field

class WriteFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")
    content: str = Field(description="The content to be written to the file")

class ReadFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")


response = client.beta.chat.completions.parse(
    model="gpt-5",
    messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
              {"role": "user", "content": "Write a file called 'test.txt' with the content 'Hello, world!'"}],
    response_format=WriteFileOperation)

output_write_file_ops = response.choices[0].message.parsed

In [6]:
print(output_write_file_ops.file_path)
print(output_write_file_ops.content)

test.txt
Hello, world!


In [ ]:
def level2_agent_llm_structured(prompt):
    response = client.beta.chat.completions.parse(
    model="gpt-5",
    messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
              {"role": "user", "content": prompt}],
    response_format=WriteFileOperation)# Structured OUTPUT from the LLM!
    output_args = response.choices[0].message.parsed
    # FUNCTION CALLING!
    write_file(output_args.file_path, output_args.content)
    print("File was created!")
    return output_args

level2_agent_llm_structured("Write a file called 'level2agentoutput.txt' with the content 'Level 2 works!'")

File was created!


WriteFileOperation(file_path='level2agentoutput.txt', content='Level 2 works!')

In [ ]:
# save this as sample_input.txt
response = get_response("Create a 3 paragraph exaplanation of modern llm agents")
write_file("sample_input.txt", response)

'Modern LLM agents are goal-directed systems built around large language models that can plan and act, not just chat. They interact with external tools, services, and environments to achieve outcomes, iterating through a loop of observing context, planning next steps, taking actions, and using the results to refine their approach. Architecturally, they pair an LLM with a function-calling layer for tools and APIs, a memory or state store, and policies that constrain behavior. This enables multi-step workflows such as analyzing datasets, booking travel, triaging support tickets, or orchestrating business processes end to end.\n\nTo stay grounded, agents often augment the model with retrieval so it can consult documents, knowledge bases, or the web before answering. Tool use spans calling APIs and databases, browsing, and writing or executing code in a sandbox to transform data or validate results. Many implementations decompose tasks into subgoals, track intermediate state, and emit stru

In [ ]:
class TaskList(BaseModel):
    tasks: list[str] = Field(description="A list of tasks to be executed")

def level3_agent_loop(task_prompt):
    task_list_prompt = f"""
    Break this task into a list of tasks as function call strings (or just strings if no function call is needed). 
    """
    task_list_response = client.beta.chat.completions.parse(
        model="gpt-5",
        messages=[{"role": "system", "content": "You take in tasks and you break them down into a python \
                   list of sub-tasks as function call strings (or just strings if no function call \
                   is needed). each element of the list should be a single function call or a string \
                   specifying a task to be done. The only allowed functions are: \
                   write_file(),\
                   read_file()"},
              {"role": "user", "content": task_prompt}],
    response_format=TaskList)# Structured OUTPUT from the LLM!
    task_list = task_list_response.choices[0].message.parsed.tasks
    
        
prompt = "Read the file: 'sample_input.txt' and write a summary of that file into a new file called 'summary_file.txt'"
level3_agent_loop(prompt)

["read_file('sample_input.txt')", 'Create a concise summary (3-5 sentences or bullet points) of the file’s main ideas, key points, and conclusions based on the content read in the previous step.', "write_file('summary_file.txt', '<insert concise summary from previous step here>')"]
